# **Brecha salarial en el sector privado de Argentina**
***

#### **Introducción**
***
En este estudio se evalúa la brecha salarial existente en Argentina dentro del sector privado, utilizando los salarios brutos informados por las empresas a la Administración Federal de Ingresos Públicos (AFIP) sobre la nómina del personal en relación de dependencia. Estos informes se realizan mediante el formulario N° 931 y tienen carácter de declaración jurada. Las mismas empresas también deben declarar su actividad principal según un sistema de clasificación conocido como Clasificador de Actividades Económicas (CLAE).  
El período analizado va desde Enero del 2007 a Junio 2022 basado en la información proporcionada a la fecha por las autoridades.  
Por otro lado, la clasificación sobre el género que realizaron en este dataset es determinada a partir de la Clave Única de Identificación Tributaria (CUIT) o Documento Nacional de Identidad (DNI).  
En consiguiente, se van a obtener datos que permitan comparar los salarios brutos entre hombres y mujeres durante todo ese período y segmentado por las distintas actividades.

#### **Consideraciones generales**
***
Cabe destacar que la clasificación binaria realizada (hombre y mujer) resulta ineficiente para cubrir la complejidad comprendida en el concepto de género. Sin embargo, a los fines de este estudio cuando se hable de brecha salarial o de género será desde este enfoque tradicional debido a que los datos recolectados se encuentran plasmados de esta forma.  

Los salarios informados son salarios brutos mensuales (exentos de aportes y deducciones  particulares) y se encuentran en pesos argentinos. Debido a lo dificultoso de interpretar los datos y visualizaciones del salario en pesos durante un período de varios años, a causa de la depreciación frente a la inflación, se van a utilizar salarios dolarizados. Dicha dolarización de los salarios no altera la brecha salarial que se obtendría al comparar salarios en pesos pero si permite una comparación en gráficas con mucha mayor claridad. La dolarización fue calculada con el valor del dólar libre o dólar blue respectivo de cada fecha para evitar confusiones al utilizar el dólar oficial en períodos de restricción cambiaria.  

De las distintas clasificaciones de actividad que hay sobre las empresas se seleccionó la CLAE2 que cuenta con más de 80 categorías perfectamente descriptas.


#### **Data**
***
Toda la información utilizada es pública, provienente del Ministerio de Economía de la Nación Argentina y obtenida por medio de la web https://datos.gob.ar/. Para este estudio en particular se utilizaron los siguientes datasets:

Percentiles de salario total empresas por género y clae2 : http://datos.produccion.gob.ar/dataset/803a2436-0240-4f87-916b-25cad0213918/archivo/a03a4818-ae25-4d49-ac77-de75980c2f9f

Diccionario de claes : http://datos.produccion.gob.ar/dataset/803a2436-0240-4f87-916b-25cad0213918/archivo/82c6602b-0138-4de5-a7c3-33e397750dec

Estos datos se obtienen en formato CSV listos para iniciar su proceso de transformación y carga en la Jupyter Notebook.

Por otro lado, para la obtención del valor del dólar libre o dólar blue histórico se utilizó la página web de Ámbito financiero https://www.ambito.com/contenidos/dolar-informal-historico.html. En este caso, el archivo obtenido fue un .xlsx de Excel al que se le realizaron operaciones de transformación mínimas para obtener el CSV de trabajo.

Si desea consultar la metodología utilizada por el Ministerio de Economía de la Nación Argentina para la realización de estas series de información puede hacerlo descargando el PDF que se encuentra en el siguiente link https://datos.gob.ar/dataset/produccion-percentiles-salarios-por-genero-actividad/archivo/produccion_3bcac8aa-7636-4e18-80d3-1f63da309ec3.

#### **Metodología**
***

Importando librerias

In [1]:
import pathlib
import pandas as pd
import numpy as np
import datetime

Generando las rutas para la correcta carga de archivos independientemente del sistema operativo y usuario

In [2]:
current_dir = pathlib.Path().resolve()
data_dir = current_dir.parent.joinpath('data', 'raw')
data_processed_dir = current_dir.parent.joinpath('data', 'processed')
# list(data_dir.glob('*'))          # With this line can list the files that you have at data/raw for check it.

In [3]:
salary_clae2_table = data_dir.joinpath('percentiles_mujer_clae2_tot_emp.csv')
historic_dollar_value_table = data_dir.joinpath('valor_dolar_historico.csv')
clae_activity_table = data_dir.joinpath('clae_agg.csv')

Carga y generación de DataFrame a partir de salarios por género y actividad (clae2) en el período 01-01-2007 al 01-06-2022

In [4]:
df_salary = pd.read_csv(salary_clae2_table, header=0, names=['date','clae2','genre','p10','p25','salary','median','p75','p90','p99'])
df_salary.drop(['p10','p25','median','p75','p90','p99'],axis=1,inplace=True)
df_salary

,date,clae2,genre,salary
0,2007-01-01,1,0,1042
1,2007-01-01,1,1,1019
2,2007-01-01,2,0,909
3,2007-01-01,2,1,1048
4,2007-01-01,3,0,3786
...,...,...,...,...
31615,2022-06-01,95,1,155848
31616,2022-06-01,96,0,145037
31617,2022-06-01,96,1,107567
31618,2022-06-01,999,0,273617


Luego de eliminar las columnas que no interesan, conviven en la misma tabla los salarios de hombres (genre=0) y mujeres (genre=1) . Posteriormente se van a separar en dos tablas para una mejor comparación. Notar también que el salario informado se encuentra en pesos argentinos.

In [5]:
df_salary.dtypes

date      object
clae2      int64
genre      int64
salary     int64
dtype: object

In [6]:
df_salary.date = df_salary.date.astype('datetime64')
df_salary.dtypes

date      datetime64[ns]
clae2              int64
genre              int64
salary             int64
dtype: object

Tras pasar la columna 'date' al formato datetime64, se genera una nueva tabla solo con las fechas del DataFrame. Esta tabla va a permitir más adelante la fusión entre los salarios y el valor del dolar histórico

In [7]:
df_date_table = df_salary.select_dtypes(include=['datetime64'])
df_date_table


,date
0,2007-01-01
1,2007-01-01
2,2007-01-01
3,2007-01-01
4,2007-01-01
...,...
31615,2022-06-01
31616,2022-06-01
31617,2022-06-01
31618,2022-06-01


In [8]:
df_date_table = df_date_table.drop_duplicates()
df_date_table = df_date_table.reset_index()
df_date_table = df_date_table.drop('index',axis=1)
df_date_table

,date
0,2007-01-01
1,2007-02-01
2,2007-03-01
3,2007-04-01
4,2007-05-01
...,...
181,2022-02-01
182,2022-03-01
183,2022-04-01
184,2022-05-01


Carga y generación de DataFrame a partir del valor del dolar libre histórico

In [9]:
df_dollar = pd.read_csv(historic_dollar_value_table, header=0, names=['date','dollar'])
df_dollar

,date,dollar
0,2022-06-01,206.00
1,2022-05-31,206.00
2,2022-05-30,206.50
3,2022-05-27,206.50
4,2022-05-26,207.00
...,...,...
3795,2007-01-08,3.10
3796,2007-01-05,3.10
3797,2007-01-04,3.10
3798,2007-01-03,3.09


In [10]:
df_dollar.date = df_dollar.date.astype('datetime64')
df_dollar.dtypes

date      datetime64[ns]
dollar           float64
dtype: object

In [11]:
df_dollar = df_dollar.sort_values('date',ascending=True)
df_dollar = df_dollar.reset_index()
df_dollar = df_dollar.drop('index',axis=1)
df_dollar

,date,dollar
0,2007-01-02,3.09
1,2007-01-03,3.09
2,2007-01-04,3.10
3,2007-01-05,3.10
4,2007-01-08,3.10
...,...,...
3795,2022-05-26,207.00
3796,2022-05-27,206.50
3797,2022-05-30,206.50
3798,2022-05-31,206.00


In [12]:
df_dollar['month'] = pd.DatetimeIndex(df_dollar['date']).month
df_dollar['year'] = pd.DatetimeIndex(df_dollar['date']).year
df_dollar

,date,dollar,month,year
0,2007-01-02,3.09,1,2007
1,2007-01-03,3.09,1,2007
2,2007-01-04,3.10,1,2007
3,2007-01-05,3.10,1,2007
4,2007-01-08,3.10,1,2007
...,...,...,...,...
3795,2022-05-26,207.00,5,2022
3796,2022-05-27,206.50,5,2022
3797,2022-05-30,206.50,5,2022
3798,2022-05-31,206.00,5,2022


Como la tabla del dolar histórico se encuentra con frecuencia diaria y los salarios se cargan de forma mensual, se calcula el promedio del dolar para cada mes. De esta manera, el resultado al dolarizar los salarios será muy cercano a la realidad. 

In [13]:
df_grouped_dollar = df_dollar.groupby(['year','month']).mean()
# df_grouped_dollar                  # This line is only for check the agrupation

/tmp/ipykernel_885/189516223.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_grouped_dollar = df_dollar.groupby(['year','month']).mean()


In [14]:
df_grouped_dollar = df_grouped_dollar.reset_index()
df_grouped_dollar

,year,month,dollar
0,2007,1,3.108182
1,2007,2,3.118000
2,2007,3,3.121364
3,2007,4,3.119474
4,2007,5,3.115909
...,...,...,...
181,2022,2,213.710526
182,2022,3,201.666667
183,2022,4,199.736842
184,2022,5,204.350000


Ahora se concatenan las tablas de fecha y la del dolar histórico para luego poder fusionar con la de salarios sin ningún inconveniente

In [15]:
df_historic_dollar = pd.concat([df_date_table, df_grouped_dollar], axis=1)
df_historic_dollar = df_historic_dollar.drop(['year','month'], axis=1)
df_historic_dollar

,date,dollar
0,2007-01-01,3.108182
1,2007-02-01,3.118000
2,2007-03-01,3.121364
3,2007-04-01,3.119474
4,2007-05-01,3.115909
...,...,...
181,2022-02-01,213.710526
182,2022-03-01,201.666667
183,2022-04-01,199.736842
184,2022-05-01,204.350000


In [16]:
df_historic_dollar.dtypes

date      datetime64[ns]
dollar           float64
dtype: object

In [17]:
df_historic_dollar['dollar'] = np.round(df_historic_dollar['dollar'], decimals = 2)
df_historic_dollar

,date,dollar
0,2007-01-01,3.11
1,2007-02-01,3.12
2,2007-03-01,3.12
3,2007-04-01,3.12
4,2007-05-01,3.12
...,...,...
181,2022-02-01,213.71
182,2022-03-01,201.67
183,2022-04-01,199.74
184,2022-05-01,204.35


Antes de fusionar la tabla final de dolar histórico junto a la de salarios se realiza la exportación de la misma como un archivo CSV. Este archivo puede utilizarse luego en herramientas de visualización como PowerBI o Tableau.

In [18]:
df_historic_dollar.to_csv(data_processed_dir.joinpath('historic_dollar.csv'), index=None)

Fusión entre la tabla de salarios y tabla dolar histórico final por su campo en común 'date'

In [19]:
df_salary = df_salary.merge(df_historic_dollar, on='date')
df_salary

,date,clae2,genre,salary,dollar
0,2007-01-01,1,0,1042,3.11
1,2007-01-01,1,1,1019,3.11
2,2007-01-01,2,0,909,3.11
3,2007-01-01,2,1,1048,3.11
4,2007-01-01,3,0,3786,3.11
...,...,...,...,...,...
31615,2022-06-01,95,1,155848,206.00
31616,2022-06-01,96,0,145037,206.00
31617,2022-06-01,96,1,107567,206.00
31618,2022-06-01,999,0,273617,206.00


Se crea una nueva columna con el salario dolarizado a partir del cálculo del salario en pesos argentinos y el valor del dolar que ahora está incorporado a la tabla

In [20]:
df_salary['dolarized_salary'] = df_salary.apply(lambda x : x['salary'] / x['dollar'],axis=1)
df_salary

,date,clae2,genre,salary,dollar,dolarized_salary
0,2007-01-01,1,0,1042,3.11,335.048232
1,2007-01-01,1,1,1019,3.11,327.652733
2,2007-01-01,2,0,909,3.11,292.282958
3,2007-01-01,2,1,1048,3.11,336.977492
4,2007-01-01,3,0,3786,3.11,1217.363344
...,...,...,...,...,...,...
31615,2022-06-01,95,1,155848,206.00,756.543689
31616,2022-06-01,96,0,145037,206.00,704.063107
31617,2022-06-01,96,1,107567,206.00,522.169903
31618,2022-06-01,999,0,273617,206.00,1328.237864


In [21]:
df_salary['dolarized_salary'] = np.round(df_salary['dolarized_salary'], decimals = 0)
df_salary.dolarized_salary = df_salary.dolarized_salary.astype('int64')
df_salary

,date,clae2,genre,salary,dollar,dolarized_salary
0,2007-01-01,1,0,1042,3.11,335
1,2007-01-01,1,1,1019,3.11,328
2,2007-01-01,2,0,909,3.11,292
3,2007-01-01,2,1,1048,3.11,337
4,2007-01-01,3,0,3786,3.11,1217
...,...,...,...,...,...,...
31615,2022-06-01,95,1,155848,206.00,757
31616,2022-06-01,96,0,145037,206.00,704
31617,2022-06-01,96,1,107567,206.00,522
31618,2022-06-01,999,0,273617,206.00,1328


Se crean dos DataFrame independientes para dividir la tabla por género

In [22]:
df_men_salary = df_salary[df_salary['genre']==0]
df_men_salary = df_men_salary.drop(['genre'],axis=1)
df_women_salary = df_salary[df_salary['genre']==1]
df_women_salary = df_women_salary.drop(['genre'],axis=1)
df_men_salary

,date,clae2,salary,dollar,dolarized_salary
0,2007-01-01,1,1042,3.11,335
2,2007-01-01,2,909,3.11,292
4,2007-01-01,3,3786,3.11,1217
6,2007-01-01,5,4171,3.11,1341
8,2007-01-01,6,11168,3.11,3591
...,...,...,...,...,...
31610,2022-06-01,93,168743,206.00,819
31612,2022-06-01,94,228577,206.00,1110
31614,2022-06-01,95,183187,206.00,889
31616,2022-06-01,96,145037,206.00,704


In [23]:
df_men_salary = df_men_salary.reset_index()
df_men_salary = df_men_salary.drop('index',axis=1)
df_women_salary = df_women_salary.reset_index()
df_women_salary = df_women_salary.drop('index',axis=1)

In [24]:
df_men_salary

,date,clae2,salary,dollar,dolarized_salary
0,2007-01-01,1,1042,3.11,335
1,2007-01-01,2,909,3.11,292
2,2007-01-01,3,3786,3.11,1217
3,2007-01-01,5,4171,3.11,1341
4,2007-01-01,6,11168,3.11,3591
...,...,...,...,...,...
15805,2022-06-01,93,168743,206.00,819
15806,2022-06-01,94,228577,206.00,1110
15807,2022-06-01,95,183187,206.00,889
15808,2022-06-01,96,145037,206.00,704


In [25]:
df_women_salary

,date,clae2,salary,dollar,dolarized_salary
0,2007-01-01,1,1019,3.11,328
1,2007-01-01,2,1048,3.11,337
2,2007-01-01,3,2514,3.11,808
3,2007-01-01,5,3457,3.11,1112
4,2007-01-01,6,6811,3.11,2190
...,...,...,...,...,...
15805,2022-06-01,93,129490,206.00,629
15806,2022-06-01,94,174699,206.00,848
15807,2022-06-01,95,155848,206.00,757
15808,2022-06-01,96,107567,206.00,522


Conversión y exportación de los DataFrames por género creados a nuevos archivos CSV. Estos CSV pueden utilizarse luego en herramientas de visualización como PowerBI o Tableau

In [26]:
df_men_salary.to_csv(data_processed_dir.joinpath('men_salary.csv'), index=None)
df_women_salary.to_csv(data_processed_dir.joinpath('women_salary.csv'), index=None)

Carga y generación de un DataFrame con el diccionario de actividades económicas

In [27]:
df_activities = pd.read_csv(clae_activity_table)
df_activities

,clae6,clae6_desc,clae3,clae3_desc,clae2,clae2_desc,letra,letra_desc
0,14211,"Cría de ganado equino, excepto la realizada en...",14,Cría de animales,1,"Agricultura, ganadería, caza y servicios relac...",A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
1,11331,Cultivo de hortalizas de hoja y de otras horta...,11,Cultivos temporales,1,"Agricultura, ganadería, caza y servicios relac...",A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
2,14410,Cría de ganado ovino -excepto en cabañas y par...,14,Cría de animales,1,"Agricultura, ganadería, caza y servicios relac...",A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
3,11211,Cultivo de soja,11,Cultivos temporales,1,"Agricultura, ganadería, caza y servicios relac...",A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
4,17010,Caza y repoblación de animales de caza (Inclu...,17,"Caza, repoblación de animales de caza y serv...",1,"Agricultura, ganadería, caza y servicios relac...",A,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y ..."
...,...,...,...,...,...,...,...,...
945,960201,Servicios de peluquería,960,Servicios personales ncp,96,Otros servicios personales,S,SERVICIOS DE ASOCIACIONES Y SERVICIOS PERSONA...
946,960202,"Servicios de tratamiento de belleza, excepto l...",960,Servicios personales ncp,96,Otros servicios personales,S,SERVICIOS DE ASOCIACIONES Y SERVICIOS PERSONA...
947,960102,"Lavado y limpieza de artículos de tela, cuero ...",960,Servicios personales ncp,96,Otros servicios personales,S,SERVICIOS DE ASOCIACIONES Y SERVICIOS PERSONA...
948,960300,Pompas fúnebres y servicios conexos,960,Servicios personales ncp,96,Otros servicios personales,S,SERVICIOS DE ASOCIACIONES Y SERVICIOS PERSONA...


In [28]:
df_activities.drop(['clae6','clae6_desc','clae3','clae3_desc','letra','letra_desc'],axis=1,inplace=True)
df_activities.columns = ['clae2', 'activity']
df_activities

,clae2,activity
0,1,"Agricultura, ganadería, caza y servicios relac..."
1,1,"Agricultura, ganadería, caza y servicios relac..."
2,1,"Agricultura, ganadería, caza y servicios relac..."
3,1,"Agricultura, ganadería, caza y servicios relac..."
4,1,"Agricultura, ganadería, caza y servicios relac..."
...,...,...
945,96,Otros servicios personales
946,96,Otros servicios personales
947,96,Otros servicios personales
948,96,Otros servicios personales


In [29]:
df_activities = df_activities.drop_duplicates()
df_activities

,clae2,activity
0,1,"Agricultura, ganadería, caza y servicios relac..."
82,2,Silvicultura y explotación forestal
89,3,Pesca y acuicultura
95,5,Extracción de carbón y lignito
97,6,Extracción de petróleo crudo y gas natural
...,...,...
914,93,"Actividades deportivas, recreativas y de entre..."
925,94,Servicios de asociaciones
934,95,Reparación de computadoras y equipos de uso do...
942,96,Otros servicios personales


In [30]:
df_activities = df_activities.reset_index()
df_activities = df_activities.drop('index',axis=1)
df_activities.head()

,clae2,activity
0,1,"Agricultura, ganadería, caza y servicios relac..."
1,2,Silvicultura y explotación forestal
2,3,Pesca y acuicultura
3,5,Extracción de carbón y lignito
4,6,Extracción de petróleo crudo y gas natural


In [31]:
df_activities.dtypes

clae2        int64
activity    object
dtype: object

In [32]:
convert = {'clae2': str, 'activity': str}
df_activities = df_activities.astype(convert)
df_activities.dtypes

clae2       object
activity    object
dtype: object

Conversión y exportación del DataFrame de actividades como archivo CSV. Este CSV puede utilizarse luego en herramientas de visualización como PowerBI o Tableau

In [33]:
df_activities.to_csv(data_processed_dir.joinpath('activities.csv'), index=None)

#### **Comparación y visualización de los datos**
***
Los datos ya procesados dieron origen a 4 archivos CSV: valor histórico del dólar, diccionario de actividades clae2, salarios dolarizados de hombres según actividad y salarios dolarizados de mujeres según actividad. Tras un correcto modelado de los datos a través de PowerBI se obtienen diversas gráficas que permiten comparar, visualizar y extraer información relevante.

En el siguiente link va a poder interactuar de forma dinámica con estas visualizaciones y sacar sus propias conclusiones.
https://app.powerbi.com/view?r=eyJrIjoiNmU3NDM4NzMtNDhmYi00YjNhLWFkNmItZGMxYTBjMGU1MDkwIiwidCI6Ijk2ZTcyZDA0LTU3MGQtNGM2YS1hNzNkLTQzMmM2Y2EwNTU0NSJ9.  
-Se recomienda utilizar una PC para interactuar de la mejor forma con la visualización aunque puede realizarse también mediante móviles-

Esta visualización le permite segmentar por todas, una o varias actividades y también segmentar por diversos períodos de tiempo (dentro del 01/2007 al 06/2022). Las gráficas van a mostrarle la comparación entre el salario bruto dolarizado de hombres y mujeres, y la evolución de la brecha salarial en el tiempo.

#### **Resultados y discusión**
***
El análisis arroja resultados que van en línea con todas las estimaciones actuales sobre la brecha salarial de género existente en Argentina. En Junio del 2022 se registra una brecha salarial en el sector privado del 16,4 %. Sin embargo, se observa una clara diferencia cuando se analizan actividades económicas en particular. 

En actividades como la Pesca y acuicultura o el Transporte acúatico la brecha salarial actual asciende a más del 50 % (57,5 % y 50,3 % respectivamente). También se observa una brecha salarial elevada en actividades de Transporte aéreo durante todo el período evaluado con un valor promedio del 35,1 %. Será motivo para otro estudio el investigar si esta situación puede deberse a la disparidad entre el salario percibido por realizar la misma tarea o a la falta de mujeres en puestos ejecutivos o de mayor importancia en este tipo de sectores, que le permitan cosechar salarios más abultados. El top de actividades con mayor brecha salarial lo completan Servicios asociados a la actividad financiera y de seguros, y Actividades de apoyo al petróleo y la minería con una brecha salarial promedio del 38,1 % y 34,7 % respectivamente.

Por otro lado, existen actividades donde las mujeres ganan más que los hombres y algunos casos son realmente llamativos. El dato más sorprendente es el de Construcción de edificios y partes donde la brecha salarial es del -39,3 % en favor de las mujeres; también en Actividades especializadas en construcción la brecha salarial es del -4,2 %. Escapa del objetivo de este informe esclarecer o poder fundamentar estos valores pero podría estar relacionado con el grado de informalidad que se maneja en este sector y en varios sectores lo cual complica el análisis del salario real percibido por los trabajadores. Dentro del top de las actividades donde las mujeres se encuentran por encima siguen Silvicultura y explotación forestal, Elaboración de productos de madera, Obras de ingeniería civil y Enseñanza con valores del -29,5 %, -14,5 %, -7,8 % y -3,2 % de brecha salarial promedio a lo largo del período evaluado.

Por último, si comparamos la brecha salarial del sector privado en Junio 2022 del 16,4 % con la que existía en Enero del 2007 del 21,8 % se observa una reducción de la misma en más de 5 puntos porcentuales que equivalen a una disminución del 24,8 %.

#### **Conclusión**
***
Teniendo en cuenta que este estudio no profundiza en la participación existente de hombres y mujeres en las diversas actividades económicas ni su distribución en cuanto a puestos que ocupan dentro de las organizaciones o empresas, es difícil determinar las causas que expliquen los números informados. Es cierto que hoy hay diversos enfoques y analizar esta problemática en detalle es sumamente complejo. Sin embargo se hace evidente que, tanto por una menor participación, por cobrar un menor salario a misma tarea realizada o por una menor cantidad de mujeres en puestos importantes o ejecutivos, las mujeres quedan rezagadas en cuanto al salario percibido plasmando la brecha salarial actual.

Otra cuestión que escapa de la incumbencia de este análisis es la informalidad existente en Argentina. El trabajo informal que se calcula está cercano al 40 % de los trabajadores totales seguramente altere estos resultados presentados y posiblemente los empeore.

A pesar de todo, es notorio destacar que en la gran mayoría de actividades hay una evolución favorable de esta brecha que se refleja en la disminución de la misma durante los 15 años comprendidos en el período 2007 al 2022. Esperemos que los factores que están influyendo positivamente, como políticas con perspectiva de género y fundamentalmente el abordamiento y una conciencia social diferente sobre estos temas, continuen ayudando para lograr lo más pronto posible un escenario de igualdad total.

**Repositorio**  

https://github.com/gastonvermeulen/brecha_salarial


***
Este informe fue realizado por  <a href='https://www.linkedin.com/in/gast%C3%B3n-vermeulen-73a93239/'> Gastón Vermeulen </a>